train classifier 

In [133]:
import csv 
import pandas as pd 

In [134]:
df = pd.read_csv('all_data.csv')

In [135]:
data = df.headline
target = df.is_clickbait

In [136]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(\
    data, target, test_size=0.20, random_state=42)

In [137]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

In [138]:
#what would happen with IDF=true?
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_train_tf = tfidf_transformer.fit_transform(X_train_counts)

In [139]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', penalty='l2'\
    ,alpha=1e-3, random_state=42,max_iter=5, tol=None).fit(X_train_tf, y_train)


In [140]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect',CountVectorizer()),
                     ('tfidf',TfidfTransformer()),
                    ('clf',SGDClassifier()),
                    ])

In [141]:
text_clf.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))])

In [142]:
import numpy as np 
predicted = text_clf.predict(X_test)

In [299]:
np.mean(predicted==y_test)

0.97015625

In [300]:
from sklearn import metrics
metrics.confusion_matrix(y_test,predicted)

array([[3092,   58],
       [ 133, 3117]])

using headlines I collected

In [301]:
file_name = 'mirror.co.uk.csv'
file = pd.read_csv('finished_csv/'+file_name)


In [302]:
X_test = file.title

In [303]:
source_predicted = text_clf.predict(X_test) 
tmp = []
for prediction in source_predicted:
    tmp.append(prediction)

In [304]:
file['is_clickbait']=tmp

In [305]:
file.to_csv('c_'+file_name,sep=',')